## 한국수출입은행 - 원화 환율(최근까지) 및 DB 저장

### 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장

In [1]:
import requests
import pandas as pd
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

In [2]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT date FROM fx_eximbank ORDER BY date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

In [3]:
# 2023년 1월 1일부터 2023년 6월 30일까지 데이터 일괄 저장
date_from = myutil.get_next_day(last_date)
date_to = myutil.get_previous_day()
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

In [4]:
date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_fx_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP01'             # 환율
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    df_fx = pd.json_normalize(response.json())
    if not df_fx.empty:
        # print(df_fx['result'].unique())
        if RESULT_SUCCESS not in df_fx['result'].unique():         # 성공이 아니라면
            print(f'{date_str}: Result is not successful')
            pass
        df_fx.drop('result', axis=1, inplace=True)
        df_fx.insert(0, 'date', date_str)
        df_fx_all = pd.concat([df_fx_all, df_fx], ignore_index=True)

In [5]:
df_fx_all.shape

(69, 11)

In [6]:
df_fx_all

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm
0,20230710,AED,352.36,359.47,355.92,355,0,0,355,355.92,아랍에미리트 디르함
1,20230710,AUD,865.51,883,874.26,874,0,0,874,874.26,호주 달러
2,20230710,BHD,"3,433.05","3,502.4","3,467.73","3,467",0,0,"3,467","3,467.73",바레인 디나르
3,20230710,BND,961.03,980.44,970.74,970,0,0,970,970.74,브루나이 달러
4,20230710,CAD,974.71,994.4,984.56,984,0,0,984,984.56,캐나다 달러
...,...,...,...,...,...,...,...,...,...,...,...
64,20230712,SAR,341.78,348.69,345.24,345,0,0,345,345.24,사우디 리얄
65,20230712,SEK,120.28,122.71,121.5,121,0,0,121,121.5,스웨덴 크로나
66,20230712,SGD,956.29,975.6,965.95,965,0,0,965,965.95,싱가포르 달러
67,20230712,THB,36.85,37.6,37.23,37,0,0,37,37.23,태국 바트


#### DB에 저장

In [22]:
df_fx_all_save = df_fx_all[['date', 'cur_unit', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r']]

In [23]:
df_fx_all_save[df_fx_all_save['cur_unit'].str.contains('USD')]

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
22,20230710,USD,"1,294.22","1,320.37","1,307.3","1,307","1,307","1,307.3"
45,20230711,USD,"1,290.76","1,316.83","1,303.8","1,303","1,303","1,303.8"
68,20230712,USD,"1,282.05","1,307.95","1,295","1,295","1,295","1,295"


* 데이터 전처리

In [26]:
# 숫자의 소수점(,) 없애기
df_fx_all_save.iloc[:,2:8] = df_fx_all_save.iloc[:,2:8].apply(lambda x: x.str.replace(',', ''), axis = 1)
# 통화코드에서 세 자리 초과하는 내용은 지우고 통화코드 생성: (예) JPY(100) -> JPY로 변경
df_fx_all_save.insert(2, 'cur_code', df_fx_all_save['cur_unit'].str[0:3])
df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [17]:
df_fx_all_save

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
0,20230710,AED,352.36,359.47,355.92,355,355,355.92
1,20230710,AUD,865.51,883,874.26,874,874,874.26
2,20230710,BHD,3433.05,3502.4,3467.73,3467,3467,3467.73
3,20230710,BND,961.03,980.44,970.74,970,970,970.74
4,20230710,CAD,974.71,994.4,984.56,984,984,984.56
...,...,...,...,...,...,...,...,...
64,20230712,SAR,341.78,348.69,345.24,345,345,345.24
65,20230712,SEK,120.28,122.71,121.5,121,121,121.5
66,20230712,SGD,956.29,975.6,965.95,965,965,965.95
67,20230712,THB,36.85,37.6,37.23,37,37,37.23


In [27]:
df_fx_all_save[df_fx_all_save['cur_code'].str.contains('USD')]

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
22,20230710,USD,1294.22,1320.37,1307.3,1307,1307,1307.3
45,20230711,USD,1290.76,1316.83,1303.8,1303,1303,1303.8
68,20230712,USD,1282.05,1307.95,1295,1295,1295,1295


* DB 테이블에 저장

In [28]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fx_all_save.itertuples():
    sql = "insert into fx_eximbank (date, cur_code, ttb, tts, deal_bas_r, bkpr, kftc_bkpr, kftc_deal_bas_r) \
                           values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
conn.commit()
conn.close()